In [ ]:
import visvis as vv
import numpy as np
import sys
import os
import subprocess
import argparse
import matplotlib.pyplot as pl

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
dir_name = 'test_mhrw_synth_02'

In [ ]:
propIdx = 0

properties = ['Density',
              'LogSusceptibility',
              'ThermalConductivity',
              'ThermalProductivity',
              'LogResistivityX',
              'LogResistivityY',
              'LogResistivityZ',
              'ResistivityPhase',
              'PWaveVelocity']
class Data(object):
    def __init__(self, inputFile):
        raw = np.load(inputFile)
        self.resolution = raw['resolution'].flatten()
        self.bounds = np.array([raw['x_bounds'], raw['y_bounds'], raw['z_bounds']])[:,:,0]
        self.nlayers = sum(1 for key in raw if key.startswith('layer'))
        self.nboundaries = sum(1 for key in raw if key.startswith('boundary'))
        self.properties = [raw[k] for k in properties]
        self.layers = [raw['layer'+str(k)]for k in range(self.nlayers)]
        self.boundaries = [raw['boundary'+str(k)]for k in range(self.nlayers)]
        self.samples = self.boundaries[0].shape[0]
        #reshape these key layers
        newshapeVox = (-1, self.resolution[2], self.resolution[1],self.resolution[0])
        newshapeSurf = (-1, self.resolution[1], self.resolution[0])
        self.layers = [k.reshape(newshapeVox, order='f') for k in self.layers]
        self.properties = [k.reshape(newshapeVox, order='f') for k in self.properties]
        self.boundaries = [k.reshape(newshapeSurf, order='f') for k in self.boundaries]
        # apply a 0/1 mask to each element of self.boundaries (which is a list of 3D arrays)
"""        self.pbound = [
            np.sum([np.nonzero(sample) for sample in layer_sample_list]) / len(layer_sample_list) 
            for layer_sample_list in self.boundaries
        ]"""


In [ ]:
dirpath = '/Users/davidkohn/dev/obsidian/output/{}'.format(dir_name)
fpath = dirpath + '/voxels/voxels{}.npz'

prob_arr = 0
n_max = 10
layer = 0

for fpath_idx in range(0, n_max):
    print(fpath_idx)
    fpath_i = fpath.format(fpath_idx)
    d = Data(fpath_i)
    X = np.linspace(d.bounds[0,0], d.bounds[0,1], d.resolution[0])
    Y = np.linspace(d.bounds[1,0], d.bounds[1,1], d.resolution[1])
    Z = np.linspace(d.bounds[2,0], d.bounds[2,1], d.resolution[2])
    for i in range(d.nlayers):
        #layer = i
        count_arr = np.zeros((len(Y), len(X), len(Z)))
        for frame in range(d.samples):
            z_value_array = d.boundaries[layer][frame]
            #C = pl.cm.jet(i/float(d.nlayers))
            #C = np.array([[[C[0], C[1], C[2]]]])
            #m = vv.surf(X, Y, z_value_array, C)
            #m = vv.surf(X, Y, Z, C)

            voxel_idx_arr = [[np.where(z_val <= Z)[0][0] for z_val in x_array] for x_array in z_value_array]
            #voxel_arr = [[Z[np.where(z_val <= Z)[0][0]] for z_val in x_array] for x_array in z_value_array]
            #m = vv.surf(X, Y, voxel_arr, C, axes = a1)
            for idx_y, y in enumerate(Y):
                for idx_x, x in enumerate(X):
                    idx_z = voxel_idx_arr[idx_y][idx_x]
                    count_arr[idx_y, idx_x, idx_z] += 1
        prob_arr = count_arr/d.samples + prob_arr
        #break
prob_arr = prob_arr/n_max
#prob_arr = prob_arr/d.nlayers

In [ ]:
app = vv.use()

In [ ]:
vv.xlabel('Eastings (m)')
vv.ylabel('Northings (m)')
vv.zlabel('Depth (m)')
a2 = vv.gca()
a2.camera.fov = 70
#a2.daspect = 1, 1, -1
t = vv.volshow(
    prob_arr,
    cm=vv.CM_JET,
    #renderStyle = 'iso'
)
vv.colorbar()
#vv.ColormapEditor(a2)
t.Draw()

In [ ]:
app.Run()

In [ ]:
fpath_i = fpath.format(fpath_idx)
d = Data(fpath_i)
for frame in range(d.samples):
    app = vv.use()
    # Surface
    vv.figure(1)
    X = np.linspace(d.bounds[0,0], d.bounds[0,1], d.resolution[0])
    Y = np.linspace(d.bounds[1,0], d.bounds[1,1], d.resolution[1])
    Z = np.linspace(d.bounds[2,0], d.bounds[2,1], d.resolution[2])
    vv.xlabel('Eastings (m)')
    vv.ylabel('Northings(m)')
    vv.zlabel('Depth (m)')
    a1 = vv.gca()
    a1.camera.fov = 70
    a1.daspect = 1, 1, -1
    for i in range(d.nlayers):
        C = pl.cm.jet(i/float(d.nlayers))
        C = np.array([[[C[0], C[1], C[2]]]])
        m = vv.surf(X, Y, d.boundaries[i][frame], C)
    # Volume
    vv.figure(2)
    # # Set labels
    vv.xlabel('Eastings (m)')
    vv.ylabel('Northings (m)')
    vv.zlabel('Depth (m)')
    a2 = vv.gca()
    a2.camera.fov = 70
    a2.daspect = 1, 1, -1
    vol = d.properties[propIdx][frame]
    t = vv.volshow(vol,cm=vv.CM_JET, renderStyle='ray')


    # Create colormap editor wibject.
    vv.ColormapEditor(a2)

    # Start app
    app.Run()
